In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import argparse
import os

from MethylCDM.utils.utils import init_environment, load_config, resolve_path
from MethylCDM.data.load_methylation import download_methylation, merge_cohort
from MethylCDM.preprocessing.process_methylation import process_methylation
from MethylCDM.constants import (
    RAW_METHYLATION_DIR, 
    PROCESSED_METHYLATION_DIR,
    METADATA_METHYLATION_DIR
)

# Mimic command-line arguments
args = {
    "project": "TCGA-BRCA",
    "config_pipeline": "pipeline.yaml",
    "config_preproc": "methylation_preproc.yaml",
    "verbose": True
}

# Load the relevant configuration files 
pipeline_cfg = load_config(args['config_pipeline'])
preproc_cfg = load_config(args['config_preproc'])

# Initialize the environment for reproducible analysis
init_environment(pipeline_cfg)

# Mimic function parameters
project = args['project']
config = preproc_cfg

In [3]:
from MethylCDM.utils.utils import load_annotations, load_beta_file
from MethylCDM.constants import (
    CONFIG_DIR,
    ANNOTATION_27K,
    ANNOTATION_450K,
    ANNOTATION_EPIC
)

from MethylCDM.preprocessing.process_methylation import (
    sample_qc,
    probe_qc,
    impute_missing,
    aggregate_genes,
    clip_beta_values
)

metadata_dir = preproc_cfg.get('download', {}).get('metadata_dir', '')
metadata_dir = resolve_path(metadata_dir, METADATA_METHYLATION_DIR)
project_metadata = os.path.join(metadata_dir, args['project'],
                                f"{args['project']}_metadata.csv")
metadata = pd.read_csv(project_metadata)

from pathlib import Path
import pandas as pd
import numpy as np
import anndata as ad
from collections import defaultdict
import os
from MethylCDM.utils.utils import load_annotations, resolve_path, load_beta_file
from MethylCDM.constants import RAW_METHYLATION_DIR
from MethylCDM.preprocessing.process_methylation import (
    process_array_methylation
)

# Resolve the project's raw data directory
raw_data_dir = config.get('download', {}).get('raw_data_dir', '')
raw_data_dir = resolve_path(raw_data_dir, RAW_METHYLATION_DIR)
project_data_dir = os.path.join(raw_data_dir, f"{project}")

# Verify the raw data exists and is not empty
if not os.path.isdir(project_data_dir):
    raise FileNotFoundError(f"Raw data directory was not found at "
                            f"{project_data_dir}.")
if not os.listdir(project_data_dir):
    raise FileNotFoundError(f"Raw data directory was empty at "
                            f"{project_data_dir}.")

# Identify and load all nested beta value .txt files
beta_files = [
    f for f in Path(project_data_dir).glob("*.level3betas.parquet")
]

# Keep the first occurrence of duplicates in the metadata (redundant)
metadata = metadata.set_index('file_name')
metadata = metadata[~metadata.index.duplicated(keep = 'first')]

# Divide samples by array type for preprocessing
array_groups = defaultdict(list)

for file in beta_files:
    fn = file.stem + ".txt"
    if fn in metadata.index:
        array_type = metadata.loc[fn, 'platform']
        array_groups[array_type].append(file)

In [4]:
from MethylCDM.utils.utils import load_cpg_matrix

In [86]:
# Preprocess sample beta values by array type
gene_matrices = []
for array_type in array_groups.keys():

    # Load the beta values and their associated annotation
    cpg_matrix = load_cpg_matrix(array_groups[array_type])
    annotation = load_annotations(array_type)

    # Preprocess the beta values into a gene-level matrix
    gene_matrix = process_array_methylation(cpg_matrix, annotation, config)
    gene_matrices.append(gene_matrix)

: 

In [ ]:
# Initialize an AnnData object for the aggregated gene-level matrix
gene_matrix = pd.concat(gene_matrices, axis = 1, join = "outer")
gene_matrix = gene_matrix.sort_index()

In [7]:
gene_matrices = []

In [10]:
array_type = "Illumina Human Methylation 450"

# Load the beta values and their associated annotation
cpg_matrix = load_cpg_matrix(array_groups[array_type])
annotation = load_annotations(array_type)

# Preprocess the beta values into a gene-level matrix
gene_matrix = process_array_methylation(cpg_matrix, annotation, config)
gene_matrices.append(gene_matrix)

In [13]:
array_type = "Illumina Human Methylation 27"

# Load the beta values and their associated annotation
cpg_matrix = load_cpg_matrix(array_groups[array_type])
annotation = load_annotations(array_type)

# Preprocess the beta values into a gene-level matrix
gene_matrix = process_array_methylation(cpg_matrix, annotation, config)
gene_matrices.append(gene_matrix)

In [15]:
gene_matrices[0]

,db68a2ca-9969-4cc6-aa7a-4ab188e28fc6.methylation_array.sesame.level3betas,4884d10c-f104-430d-bbca-871fb9c8e72b.methylation_array.sesame.level3betas,79fea483-3b0a-4ae1-8917-263288975f08.methylation_array.sesame.level3betas,450f02e4-00b7-445a-adb7-145b6deacbb3.methylation_array.sesame.level3betas,7a50fb24-e9f3-4297-a1bf-6bc2e5507a55.methylation_array.sesame.level3betas,b59fc222-6929-4734-9d37-e962d861d3fc.methylation_array.sesame.level3betas,e00449ae-4e8c-444c-8651-d5806bc707fb.methylation_array.sesame.level3betas,335d7410-3034-4c91-8acb-35b144cb8ced.methylation_array.sesame.level3betas,05effd3b-1633-4100-a062-268a5da0a33a.methylation_array.sesame.level3betas,ef9ed8c4-f5d0-4d4e-85a2-24ea01ee81da.methylation_array.sesame.level3betas,...,55c9e38d-e2b4-46e3-a013-dc4a32d24cfa.methylation_array.sesame.level3betas,05f5cd04-968b-40f7-94a0-fc68042e4df7.methylation_array.sesame.level3betas,6eaeaa1d-eb70-4ad1-8f3a-129f83ba1210.methylation_array.sesame.level3betas,e1a575e6-fb64-4037-a0d3-d24c252cd236.methylation_array.sesame.level3betas,1cebd084-98a1-4649-86c2-4661155b43be.methylation_array.sesame.level3betas,7d118736-8504-45ac-8b08-14278223d47f.methylation_array.sesame.level3betas,f9b7a8db-4e15-436b-b28d-1f1983aa2707.methylation_array.sesame.level3betas,7a2635b6-159b-4b20-9313-c81e4792b19a.methylation_array.sesame.level3betas,f479919f-8309-4dc2-8297-31000ca1be6b.methylation_array.sesame.level3betas,507bfebd-1bc6-493f-8373-0c47c6314563.methylation_array.sesame.level3betas
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.712150,0.669901,0.686000,0.680160,0.686303,0.639625,0.777117,0.801391,0.709405,0.579806,...,0.791864,0.762119,0.598249,0.651670,0.759010,0.553088,0.789387,0.759767,0.780089,0.580207
A2BP1,0.358492,0.482069,0.623947,0.253041,0.420816,0.623761,0.571585,0.293161,0.646158,0.438645,...,0.482602,0.626658,0.399922,0.648419,0.412439,0.563070,0.560398,0.591088,0.652128,0.444102
A2LD1,0.753499,0.862333,0.868273,0.720557,0.629811,0.854155,0.734314,0.792057,0.856099,0.826996,...,0.832980,0.726589,0.750726,0.789741,0.917470,0.703468,0.838274,0.775646,0.886455,0.753556
A2M,0.543440,0.714239,0.775221,0.783053,0.806781,0.723715,0.906525,0.834365,0.716151,0.614019,...,0.885305,0.864946,0.527066,0.826143,0.789184,0.476898,0.773178,0.801225,0.783135,0.731472
A2ML1,0.811931,0.889701,0.809010,0.916120,0.914137,0.797072,0.926827,0.927688,0.793781,0.755559,...,0.922232,0.912283,0.636495,0.886937,0.526193,0.830436,0.801551,0.911780,0.829743,0.916418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYX,0.238622,0.269908,0.267433,0.279914,0.293789,0.255855,0.321636,0.304982,0.244697,0.236865,...,0.283404,0.250968,0.265061,0.285906,0.264724,0.192516,0.254464,0.294056,0.274893,0.265701
ZZEF1,0.535203,0.520213,0.504045,0.514576,0.547424,0.485178,0.554368,0.546329,0.505526,0.506380,...,0.544247,0.532091,0.512412,0.535378,0.453740,0.460813,0.531249,0.523890,0.508373,0.511802
ZZZ3,0.113067,0.121692,0.114833,0.125727,0.115698,0.114762,0.113501,0.125729,0.117461,0.109651,...,0.131910,0.129422,0.109360,0.119017,0.103179,0.101492,0.118013,0.115710,0.117823,0.112772


In [14]:
gene_matrices[1]

,e88f0201-1dac-4c96-bdd4-8a1d9f0671b3.methylation_array.sesame.level3betas,73c6d2b5-8329-4a75-a5d6-960950c84236.methylation_array.sesame.level3betas,21197827-03dc-4b30-8e27-37f310a6c218.methylation_array.sesame.level3betas,20f5ce9d-da3c-462e-a82d-3e0d0c8d28ed.methylation_array.sesame.level3betas,94c367f5-e0b3-4afb-a2cf-049f13adc394.methylation_array.sesame.level3betas,a8b1fd15-aee6-4d09-80e7-83c50f3dff75.methylation_array.sesame.level3betas,cbee3967-7557-4601-9e40-957a812abd8f.methylation_array.sesame.level3betas,aa7994b6-8924-4e0c-b074-df337441195e.methylation_array.sesame.level3betas,1662818b-33cc-479b-a7c3-df06841deeed.methylation_array.sesame.level3betas,4406be42-7542-48fb-a4ba-daedb867d839.methylation_array.sesame.level3betas,...,fe55c8bd-c1c2-48ba-a34a-eaf7033a7552.methylation_array.sesame.level3betas,9f7c5d86-e6b2-41ed-89ae-9f8107a56dd6.methylation_array.sesame.level3betas,1355d482-0ab9-424e-95b5-3e9ef1f136ee.methylation_array.sesame.level3betas,5a4e638b-b534-420f-9675-31cb8d4c50bb.methylation_array.sesame.level3betas,5cd576b8-248a-49e2-ac82-d877843b204a.methylation_array.sesame.level3betas,5abd7590-46fd-4016-b38a-e334a7066e25.methylation_array.sesame.level3betas,438c261c-0a78-4fb2-91bf-6935e4e27e31.methylation_array.sesame.level3betas,93655b20-9109-4c7e-afd7-7bc76ca61578.methylation_array.sesame.level3betas,e42dfb96-21de-4d42-b2a3-47a97843ebc6.methylation_array.sesame.level3betas,3ce8337c-3d78-4224-ba1b-55200e5568f1.methylation_array.sesame.level3betas
gene,,,,,,,,,,,,,,,,,,,,,
7A5,0.047710,0.150160,0.138077,0.120058,0.085341,0.069069,0.068928,0.076019,0.246651,0.047853,...,0.112596,0.122489,0.082463,0.071728,0.084036,0.097945,0.157772,0.101330,0.118966,0.062875
A1BG,0.985026,0.985521,0.977994,0.920455,0.978698,0.957954,0.975063,0.985287,0.961119,0.983581,...,0.977526,0.980961,0.982769,0.970264,0.982217,0.984684,0.986171,0.981100,0.974936,0.977719
A2BP1,0.333144,0.056916,0.021137,0.021997,0.289260,0.633633,0.125814,0.016345,0.037236,0.109890,...,0.034291,0.111390,0.159230,0.055904,0.030950,0.149810,0.642516,0.120169,0.023507,0.057766
A2ML1,0.427488,0.796957,0.411657,0.433706,0.770632,0.497578,0.788018,0.515898,0.613980,0.780578,...,0.226949,0.843007,0.897764,0.753810,0.649643,0.297891,0.367812,0.798206,0.731005,0.662091
A4GALT,0.233130,0.337138,0.266824,0.334216,0.249841,0.145369,0.151337,0.152420,0.329025,0.428770,...,0.077455,0.253833,0.394838,0.232161,0.149284,0.542182,0.652076,0.480791,0.302595,0.405871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZZ3,0.021429,0.013006,0.027005,0.016664,0.018785,0.025475,0.023148,0.020384,0.018745,0.018117,...,0.020764,0.035617,0.021760,0.020439,0.021099,0.017466,0.020834,0.015974,0.025099,0.018489
hCAP-D3,0.051813,0.040214,0.031752,0.106015,0.047511,0.045726,0.028857,0.025107,0.029403,0.022771,...,0.043869,0.038904,0.054880,0.035870,0.024789,0.038524,0.023470,0.041497,0.038273,0.096081
hCAP-H2,0.010176,0.008565,0.010492,0.007993,0.009413,0.012284,0.013068,0.011546,0.010551,0.014924,...,0.007322,0.010365,0.021920,0.012770,0.008896,0.010950,0.017951,0.010601,0.009551,0.010241
